In [6]:
import os
#import json
from dotenv import load_dotenv
#
from langchain_ollama import ChatOllama #, OllamaLLM
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
#
import streamlit as st
#
#from vectorize_documents import get_embeddings, vectordb_directory

In [5]:
load_dotenv()

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
# LangSmith tracking
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

In [4]:
vectordb_directory = 'vectordb_dir'
working_dir = os.path.dirname(f'E:{os.sep}Progi{os.sep}PythonProg{os.sep}NLP{os.sep}RAG{os.sep}Siddhardhan{os.sep}')
vectorstore_directory = f'{working_dir}{os.sep}{vectordb_directory}'
vectorstore_directory

'E:\\Progi\\PythonProg\\NLP\\RAG\\Siddhardhan\\vectordb_dir'

In [7]:
def get_embeddings():
    #model_id = 'sentence-transformers/all-MiniLM-l6-v2'
    model_id = 'BAAI/bge-m3'

    #model_args = {'device': 'cuda'}
    model_args = {'device': 'cpu'}

    embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_id,
        model_kwargs=model_args,
        encode_kwargs={'normalize_embeddings': True}
    )
    return embeddings

In [9]:
def get_vectorstore():
    huggingface_embeddings = get_embeddings()
    
    vectorstore = Chroma(
        persist_directory=vectorstore_directory,
        embedding_function=huggingface_embeddings
    )
    return vectorstore

In [10]:
vectorstore = get_vectorstore()
retriever = vectorstore.as_retriever()

e:\Progi\PythonProg\NLP\RAG\Siddhardhan\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


e:\Progi\PythonProg\NLP\RAG\Siddhardhan\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Андрей\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
#llm = OllamaLLM(model='sutyrin/saiga_mistral_7b', temperature=0.1, base_url='http://localhost:11434')
llm = ChatOllama(model='sutyrin/saiga_mistral_7b', temperature=0.1, base_url='http://localhost:11434')

In [12]:
promt_template = '''
Ты - Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.
Используй следующие фрагменты релевантного контекста, чтобы ответить на вопрос.
Если ты не знаешь ответ, просто скажи, что не знаю.
Используй максимум двадцать предложений и сделай ответ кратким.
Question: {question}
Context: {context}
Answer:
'''

In [14]:
prompt = ChatPromptTemplate.from_template(promt_template)
output_parser = StrOutputParser()
rag_chain = (prompt | llm | output_parser)

In [15]:
rag_chain.invoke({
    'context': retriever,
    'question': 'Почетные звания Томского политехнического университета?'
})

Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)
Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)
Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)
Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)


'Почетные звания Томского политехнического университета включают в себя такие категории, как "Заслуги перед университетом", "Заслуги перед наукой и техникой", "Заслуги перед образованием" и другие.'

In [16]:
rag_chain.invoke({
    'context': retriever,
    'question': 'Бесплатная выдача молока?'
})

Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)
Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)
Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)
Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)


'Не знаю, что такое бесплатная выдача молока.'

In [17]:
rag_chain.invoke({
    'context': retriever,
    'question': 'Завышение требований к экзаменам и зачетам?'
})

Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)
Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)
Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)
Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)


'Завышение требований к экзаменам и зачетам может привести к негативным последствиям, таким как стресс среди студентов, ухудшение их психического здоровья и понижение мотивации. Кроме того, это может привести к несправедливому распределению оценок и затруднениям в процессе обучения.'